In [1]:
from tn import *
import matplotlib.pyplot as plt
import numpy as np

cuda:0


In [2]:
def f(x):
    return x * np.sin(200 * (x + 1)**2)
def g(x):
    return np.sin(250 * x ** 3) * np.cos(150 * x)
def h(x):
    return np.sin(1000 * x ** 3)

In [3]:
x = np.linspace(0, 1, 1000)
yF = f(x)
yG = g(x)
yH = h(x)

In [4]:
len([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

14

In [5]:
data = []

G1 = nx.Graph()
G1.add_edges_from([[1, 2], [2, 3], [3, 1]])
nx.draw(G1, with_labels=True)
plt.savefig("results/comp-als/graph-1.png", format="PNG")
plt.clf()

data.append([G1, [10, 10, 10], [5, 5, 5]])

G2 = nx.Graph()
G2.add_edges_from([[1, 2], [2, 3], [3, 4]])
nx.draw(G2, with_labels=True)
plt.savefig("results/comp-als/graph-2.png", format="PNG")
plt.clf()
data.append([G2, [10, 10, 10, 10], [10, 10, 10]])

G3 = nx.Graph()
G3.add_edges_from([[1, 2], [2, 3], [3, 4], [4, 5], [5, 1]])
nx.draw(G3, with_labels=True)
plt.savefig("results/comp-als/graph-3.png", format="PNG")
plt.clf()
data.append([G3, [10, 10, 10, 10, 10], [7, 7, 7, 7, 7]])

G4 = nx.Graph()
G4.add_edges_from([[1, 2], [2, 3], [3, 4], [4, 5], [5, 1], [3, 5]])
nx.draw(G4, with_labels=True)
plt.savefig("results/comp-als/graph-4.png", format="PNG")
plt.clf()
data.append([G4, [10, 10, 10, 10, 10], [7, 7, 7, 7, 7, 7]])


G5 = nx.Graph()
G5.add_edges_from([[1, 2], [1, 3], [2, 4], [2, 5]])
nx.draw(G5, with_labels=True)
plt.savefig("results/comp-als/graph-5.png", format="PNG")
plt.clf()
data.append([G5, [10, 10, 10, 10, 10], [12, 12, 12, 12]])

G6 = nx.Graph()
G6.add_edges_from([[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [2, 3], [2, 4], [2, 5], [2, 6], [3, 4], [3, 5], [3, 6], [4, 5], [4, 6], [5, 6]])
nx.draw(G6, with_labels=True)
plt.savefig("results/comp-als/graph-6.png", format="PNG")
plt.clf()
data.append([G6, [10, 10, 10, 10, 10, 10], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])

i = 1
with open("results/comp-als/data.txt", "w") as file:
    for G, shape, ranks in data:
        x = np.linspace(0, 1, np.prod(shape))
        yF = f(x)
        yG = g(x)
        yH = h(x)
    
        T = [
            torch.reshape(torch.tensor(yF, dtype=torch.float), shape).to(device),
            torch.reshape(torch.tensor(yG, dtype=torch.float), shape).to(device),
            torch.reshape(torch.tensor(yH, dtype=torch.float), shape).to(device)
        ]
        j = 1
        file.write("==================== G" + str(i) + " =========================\n")
        tn = TN(G, shape, ranks)
        file.write("CR: " + str(torch.numel(T[0]) / tn.get_tn_size()) + "\n")
        file.write("Shape: " + str(shape) + "    Ranks: " + str(ranks) + "\n\n")
        
        del tn
        for t in T:
            tn = TN(G, shape, ranks)
            
            print("TN-ALS")
            start_time = time.time()
            rel_err, graph = TN.als(tn, t, 0, print_iters=True, max_time=5)
            end_time = time.time()
            elapsed_time = end_time - start_time
            print("Rel err: " + str(rel_err))
            print("Elapsed time: " + str(elapsed_time))
            print()
            file.write("TN-ALS (" + str(j) + "): err: " + str(rel_err) + "; time: " + str(elapsed_time) + "\n")

            plt.clf()
            plt.ylim(0, 0.4)
            plt.xlim(1, 100)
            plt.xlabel("Iterations")
            plt.ylabel("Rel. error")
            plt.xticks(range(0, 101, 10))
            
            plt.plot(graph[1], label="tn-als")
            del tn
            tn = TN(G, shape, ranks)
    
            print("Backprop")
            start_time = time.time()
            rel_err, graph = TN.als_grad(tn, t, print_iters=True, max_time=5)
            end_time = time.time()
            elapsed_time = end_time - start_time
            print("Rel err: " + str(rel_err))
            print("Elapsed time: " + str(elapsed_time))
            print()
            
            file.write("Backprop (" + str(j) + "): err: " + str(rel_err) + "; time: " + str(elapsed_time) + "\n")
            plt.plot(graph[1], label="backprop")
    
            plt.legend()
            plt.savefig('results/comp-als/G' + str(i) + "-T" + str(j) + ".png")
            plt.show()
            del tn
            j += 1
        i += 1

torch.Size([10, 5, 5])
torch.Size([10, 5, 5])
torch.Size([10, 5, 5])
1000
torch.Size([10, 5, 5])
torch.Size([10, 5, 5])
torch.Size([10, 5, 5])
750


<Figure size 640x480 with 0 Axes>